# M2 SETTLING TIME ANALYSIS

In [ ]:
# %matplotlib widget
import lsst_efd_client
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
global suptitle_size, axtitle_size, axlabel_size, axticks_size
suptitle_size = 16
axtitle_size = 14
axlabel_size = 12
axticks_size = 12

In [ ]:
async def get_query_df(
    efd_client: lsst_efd_client, topic: str, start_time: Time, end_time: Time
):
    """Function to get the query dataframe from the EFD.

    Args:
        efd_client (lsst_efd_client): EFD client object to use for querying.
        topic (str): EFD topic to query.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
    Returns:
        pandas.DataFrame: DataFrame containing the query results.
    """

    topic_fields = await efd_client.get_fields(topic)
    query = efd_client.build_time_range_query(
        topic, topic_fields, start_time, end_time
    )
    query_df = await efd_client.influx_client.query(query)

    return query_df

In [ ]:
async def get_merge_query(
    efd_client: lsst_efd_client.EfdClient,
    topics: list,
    start_time: Time,
    end_time: Time,
    tolerance: float = None,
    direction: str = None,
):
    """Function to merge multiple topics into a single dataframe.

    Args:
        efd_client (lsst_efd_client.EfdClient): EFD client object.
        topics (list): EFD topics to query and merge.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
        tolerance (float, optional): Tolerance time for matching, see pandas.merge_asof for more information. Defaults to None will use defualt value.
        direction (str, optional): Join method to pass to pd.merge_asod method. Defaults to None will use default value.

    Returns:
        pd.DataFrame: Merged dataframe.
    """

    query_df = list()
    for topic in topics:
        topic_fields = await efd_client.get_fields(topic)
        query = efd_client.build_time_range_query(
            topic, topic_fields, start_time, end_time
        )
        data_query = await efd_client.influx_client.query(query)
        if len(data_query) == 0:
            print(f"{topic} is not present.")
        else:
            query_df.append(data_query)
    if len(query_df) == 1:
        return query_df[0]
    elif len(query_df) == 0:
        print("No Dataframe retrieve")
        return query_df
    query_df.sort(key=lambda el: len(el), reverse=True)
    merge_df = query_df[0].copy()
    for i, df in enumerate(query_df[1:]):
        col_left = topics[i].split(".")[-1]
        col_right = topics[i + 1].split(".")[-1]
        if tolerance is None and direction is None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction="nearest",
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )
        elif tolerance is None and direction is not None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction=direction,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )
        elif tolerance is not None and direction is None:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction="nearest",
                tolerance=tolerance,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )
        else:
            merge_df = lsst_efd_client.rendezvous_dataframes(
                merge_df,
                df,
                direction=direction,
                tolerance=tolerance,
                suffixes=[f"_{col_left}", f"_{col_right}"],
            )
    return merge_df

In [ ]:
async def find_start_stop_slew(
    efd_client: lsst_efd_client, start_time: Time, end_time: Time, axis: str
):
    """Function to find the start and stop times of slews.

    Args:
        efd_client (lsst_efd_client): EFD client object to use for querying.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
        axis (str): Axis to find the slews for (elevation or azimuth).

    Returns:
        dict: dictionary containing the start and stop times of each slew.
    """

    if axis == "elevation":
        topic = "lsst.sal.MTMount.logevent_elevationInPosition"
    elif axis == "azimuth":
        topic = "lsst.sal.MTMount.logevent_azimuthInPosition"

    el_in_position_df = await get_query_df(
        efd_client=efd_client,
        topic=topic,
        start_time=start_time,
        end_time=end_time,
    )
    el_in_position_df["movement"] = 0

    c = 0
    for index, row in el_in_position_df.iterrows():
        if not row["inPosition"]:
            c += 1
        el_in_position_df.loc[index, "movement"] = c

    start_stop = el_in_position_df.groupby(by=["movement"]).groups
    return start_stop

In [ ]:
def plot_overview(el_df: pd.DataFrame, az_df: pd.DataFrame):
    """Function to plot the overview of the elevation and azimuth slews.

    Args:
        el_df (pandas.DataFrame): DataFrame containing the elevation data.
        az_df (pandas.DataFrame): DataFrame containing the azimuth data.

    Returns:
        None
    """

    fig = plt.figure(layout="constrained")
    ax = fig.add_subplot(111)
    el_df["actualPosition"].plot(
        ax=ax, label="elevation", color="blue", fontsize=axticks_size
    )
    az_df["actualPosition"].plot(
        ax=ax, label="azimuth", color="green", fontsize=axticks_size
    )

    ax.legend()
    ax.set_xlabel("Time UTC", fontsize=axlabel_size)
    ax.set_ylabel("Elevation/Azimuth, deg", fontsize=axlabel_size)
    # fig.savefig(
    #     r"C:\Users\utente\Desktop\PhD\LSST\M2\M2_settling_time\brian\overview_el-az.png",
    #     dpi=150,
    # )

In [ ]:
efd_client = lsst_efd_client.EfdClient(efd_name="usdf_efd")

## ELEVATION

In [ ]:
axis = "elevation"
time_windows = {
    1: [
        Time("2024-04-21T03:00:00", scale="utc", format="isot"),
        Time("2024-04-21T04:20:00", scale="utc", format="isot"),
    ],
}

start_time, end_time = time_windows[1][0], time_windows[1][1]

start_stop_el = await find_start_stop_slew(
    efd_client=efd_client, start_time=start_time, end_time=end_time, axis=axis
)
start_stop_az = await find_start_stop_slew(
    efd_client=efd_client,
    start_time=start_time,
    end_time=end_time,
    axis="azimuth",
)
ints = np.array([el for el in start_stop_el.values()])

### Movement overview

In [ ]:
el_topic = "lsst.sal.MTMount.elevation"
az_topic = "lsst.sal.MTMount.azimuth"


elevation_df = await get_query_df(
    efd_client=efd_client,
    topic=el_topic,
    start_time=time_windows[1][0],
    end_time=time_windows[1][1],
)
azimuth_df = await get_query_df(
    efd_client=efd_client,
    topic=az_topic,
    start_time=time_windows[1][0],
    end_time=time_windows[1][1],
)


plot_overview(el_df=elevation_df, az_df=azimuth_df)

### TANGENT FORCE ERROR

In [ ]:
topic = "lsst.sal.MTM2.forceErrorTangent"
tangent_ferror_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=start_time,
    end_time=end_time,
)

In [ ]:
eval_act = ["force1", "force2", "force4", "force5"]
for act in eval_act:
    tr_int = []
    outlier = []

    for i, (ti, tf) in enumerate(zip(ints[:-1, 1], ints[1:, 0])):
        mn = tangent_ferror_df.loc[ti:tf, act].mean()
        std = tangent_ferror_df.loc[ti:tf, act].std()
        _mask = (tangent_ferror_df.loc[ti:tf, act] > mn + 3 * std) | (
            tangent_ferror_df.loc[ti:tf, act] < mn - 3 * std
        )
        if all(value == False for value in _mask.values):
            tr_int.append(0.0)
            continue

        tr = tangent_ferror_df.loc[ti:tf, act][_mask]
        id = tangent_ferror_df.index.get_loc(tr.index[-1])
        it = (tangent_ferror_df.index[id + 1] - tr.index[0]).total_seconds()

        if it > 5.0:
            outlier.append((i, tangent_ferror_df.index[id + 1], tr.index[0]))
            continue

        tr_int.append(it)

    fig = plt.figure(layout="constrained")
    fig2 = plt.figure(layout="constrained")

    ax = fig.add_subplot(111)
    ax.set_title(
        f"A{int(act[-1]) + 1} individual tangent force error",
        fontsize=axtitle_size,
    )
    ax.plot(tr_int)
    ax.set_xlabel("ith slew", fontsize=axlabel_size)
    ax.set_ylabel("Settling Time, s", fontsize=axlabel_size)
    ax.set_xticks(
        ax.get_xticks(), labels=ax.get_xticklabels(), fontsize=axticks_size
    )
    ax.set_yticks(
        ax.get_yticks(), labels=ax.get_yticklabels(), fontsize=axticks_size
    )
    ax.set_xlim(0.0, len(tr_int))
    # fig.savefig(f"./brian/tangent_ferror_{act}.png", dpi=150)

    ax2 = fig2.add_subplot(111)
    ax2.set_title(
        f"A{int(act[-1]) + 1} individual tangent force error",
        fontsize=axtitle_size,
    )
    hist = np.histogram(tr_int, bins=20, range=(0.0, round(max(tr_int), 1)))
    ax2.bar(hist[1][:-1], hist[0], width=hist[1][1] - hist[1][0] / 4)
    ax2.set_ylabel("Slew counts", fontsize=axlabel_size)
    ax2.set_xlabel("Settling Time, s", fontsize=axlabel_size)

    ax2.set_xticks(
        ax2.get_xticks(), labels=ax2.get_xticklabels(), fontsize=axticks_size
    )
    ax2.set_yticks(
        ax2.get_yticks(), labels=ax2.get_yticklabels(), fontsize=axticks_size
    )
    # fig2.savefig(f"./brian/tangent_ferror_{act}_bin.png", dpi=150)

## INSPECTION OF A SINGLE EVENT AND HOW IT IS MEASURED

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("A2 measured force", fontsize=axtitle_size)

which = 0
tangent_ferror_df.loc[ints[which, 1] : ints[which + 1, 0], "force2"].plot(
    ax=ax, marker=".", label=""
)
mn = tangent_ferror_df.loc[
    ints[which, 1] : ints[which + 1, 0], "force2"
].mean()
std = tangent_ferror_df.loc[
    ints[which, 1] : ints[which + 1, 0], "force2"
].std()
_mask = (
    tangent_ferror_df.loc[ints[which, 1] : ints[which + 1, 0], "force2"]
    > mn + 3 * std
) | (
    tangent_ferror_df.loc[ints[which, 1] : ints[which + 1, 0], "force2"]
    < mn - 3 * std
)
tr = tangent_ferror_df.loc[ints[which, 1] : ints[which + 1, 0], "force2"][
    _mask
]
id = tangent_ferror_df.index.get_loc(tr.index[-1])

tr_int = (tangent_ferror_df.index[id + 1] - tr.index[0]).total_seconds()

tr.plot(ax=ax, marker="x", lw=0, color="red", label="", fontsize=axticks_size)
tangent_ferror_df.force2.iloc[id : id + 2].plot(
    ax=ax,
    marker="x",
    lw=0,
    color="red",
    label="Transient",
    fontsize=axticks_size,
)
ax.set_ylabel("Measured Force, N", fontsize=axlabel_size)
ax.legend()

# ESTIMATION FOR 100% VELOCITY (ELEVATION)

In [ ]:
time_windows = {
    20: [
        Time("2023-11-16T03:23:30", scale="utc", format="isot"),
        Time("2023-11-16T03:25:30", scale="utc", format="isot"),
    ],
    30: [
        Time("2023-11-16T03:46:14", scale="utc", format="isot"),
        Time("2023-11-16T03:47:30", scale="utc", format="isot"),
    ],
    40: [
        Time("2023-11-16T06:48:00", scale="utc", format="isot"),
        Time("2023-11-16T06:50:00", scale="utc", format="isot"),
    ],
    60: [
        Time("2023-11-18T23:12:45", scale="utc", format="isot"),
        Time("2023-11-18T23:15:00", scale="utc", format="isot"),
    ],
    80: [
        Time("2023-11-19T00:50:50", scale="utc", format="isot"),
        Time("2023-11-19T00:53:00", scale="utc", format="isot"),
    ],
}

In [ ]:
topic = ["lsst.sal.MTM2.forceErrorTangent", "lsst.sal.MTMount.elevation"]
tangent_ferror_df = await get_merge_query(
    efd_client=efd_client,
    topics=topic,
    start_time=time_windows[80][0],
    end_time=time_windows[80][1],
)

In [ ]:
topic = ["lsst.sal.MTM2.forceErrorTangent", "lsst.sal.MTMount.elevation"]
for act in eval_act:

    tr_int_stat = []
    for start, end in time_windows.values():

        tangent_ferror_df = await get_merge_query(
            efd_client=efd_client,
            topics=topic,
            start_time=start,
            end_time=end,
        )

        start_stop_el = await find_start_stop_slew(
            efd_client=efd_client,
            start_time=start,
            end_time=end,
            axis="elevation",
        )

        ints = np.array([el for el in start_stop_el.values()])
        tr_int = []
        outlier = []

        for i, (ti, tf) in enumerate(zip(ints[:-1, 1], ints[1:, 0])):
            mn = tangent_ferror_df.loc[ti:tf, act].mean()
            std = tangent_ferror_df.loc[ti:tf, act].std()
            _mask = (tangent_ferror_df.loc[ti:tf, act] > mn + 3 * std) | (
                tangent_ferror_df.loc[ti:tf, act] < mn - 3 * std
            )
            if all(value == False for value in _mask.values):
                tr_int.append(0.0)
                continue

            tr = tangent_ferror_df.loc[ti:tf, act][_mask]
            id = tangent_ferror_df.index.get_loc(tr.index[-1])
            it = (
                tangent_ferror_df.index[id + 1] - tr.index[0]
            ).total_seconds()

            if it > 5.0:
                outlier.append(
                    (i, tangent_ferror_df.index[id + 1], tr.index[0])
                )
                continue
            acc = (
                tangent_ferror_df.loc[ti:tf].actualVelocity.diff()
                / tangent_ferror_df.loc[ti:tf].index.diff().total_seconds()
            ).max()
            tr_int.append((it, acc))
        tr_int = np.array(tr_int)
        tr_int_stat.append(
            (
                np.mean(tr_int[:, 0]),
                max(tr_int[:, 0]),
                min(tr_int[:, 0]),
                np.max(tr_int[:, 1]),
            )
        )
    tr_int_stat = np.array(tr_int_stat)
    x = np.array(list(time_windows.keys()))

    m, q = np.linalg.lstsq(
        np.vstack((x, np.ones_like(x))).T, tr_int_stat[:, 0], rcond=None
    )[0]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(
        f"A{int(act[-1]) + 1} Settling Time vs. Elevation speed",
        fontsize=axtitle_size,
    )
    sc = ax.scatter(
        x=x,
        y=tr_int_stat[:, 0],
        c=tr_int_stat[:, 3],
        marker="s",
        zorder=100,
    )
    ax.errorbar(
        x=x,
        y=tr_int_stat[:, 0],
        yerr=(
            tr_int_stat[:, 2],
            tr_int_stat[:, 1],
        ),
        marker="^",
        c="gray",
        lw=0,
        elinewidth=1,
    )
    ax.plot(
        np.append(x, 100.0),
        m * np.append(x, 100.0) + q,
        c="gray",
        ls="--",
        label=f"Fit, m={m:.4f}",
    )
    ax.scatter(
        100.0,
        m * 100.0 + q,
        c="red",
        marker="X",
        zorder=100,
        label="Estimated",
    )
    cbar = fig.colorbar(sc, ax=ax, label="Max Acceleration, $deg/s^2$")
    ax.set_xlabel("Elevation rate speed, %", fontsize=axlabel_size)
    ax.set_ylabel("Settling Time, s", fontsize=axlabel_size)
    ax.legend()

    fig.savefig(
        r"C:\Users\utente\Desktop\PhD\LSST\M2\M2_settling_time\brian"
        + f"\\settling_100speed_{act}.png",
        dpi=150,
    )